# Dynamics

This file contains the functions for moving the particles. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py
import os
import matplotlib.animation as animation

from scipy.integrate import odeint

plt.rcParams['axes.grid'] = True
plt.style.use('dark_background')

In [ ]:
G = 6.67259e-11  # (km**3/kg/s**2)
Ms = 1.98847e30 # solar mass in kg

In [2]:
def grav_force(mass_1, mass_2, r):
    # provides the force as a (x,y,z) array 
    # must be given two masses and a (x,y,z) distance array
    return r*(G*mass_1*mass_2)/(np.linalg.norm(r)**3)

def get_r(pos1, pos2):
    # provides the (x,y,z) distance array
    # requires pos1 and pos2 to be (x,y,z) arrays
    r_vec = np.array([pos1[0]-pos2[0], pos1[1]-pos2[1], pos1[2]-pos2[2]])
    return r_vec

def gwpl(m1, m2, r):
    """Gravitational wave power loss. 
    r must be a (x,y,z) array 
    P is power """
    P = ((G**4)*(m1+m2)*(m1*m2)**2)/((c*np.linalg.norm(r))**5)
    return P

def leapfrogintegrate(acc1, acc2, pos1, pos2, vel1, vel2, mass1, mass2):
    # pos1 = [x,y,z] array
    # pos2 = [x,y,z] array
    # vel1 = [x,y,z] array
    # vel2 = [x,y,z] array
    
    # does leapfrog integration and returns position and velocity values 
    
    N = 5000
    dt = 0.1
    
    # Create empty arrays 
    x1 = np.zeros(N)
    y1 = np.zeros(N)
    z1 = np.zeros(N)
    x2 = np.zeros(N)
    y2 = np.zeros(N)
    z2 = np.zeros(N)
    vx1 = np.zeros(N)
    vy1 = np.zeros(N)
    vz1 = np.zeros(N)
    vx2 = np.zeros(N)
    vy2 = np.zeros(N)
    vz2 = np.zeros(N)
    
    # initialize 
    for index in range(N-1):
        # set initial step 
        if index == 0:
            x1[index] = pos1[0]
            x2[index] = pos2[0]
            y1[index] = pos1[1]
            y2[index] = pos2[1]
            z1[index] = pos1[2]
            z2[index] = pos2[2]
            vx1[index] = vel1[0]
            vx2[index] = vel2[0]
            vy1[index] = vel1[1]
            vy2[index] = vel2[1]
            vz1[index] = vel1[2]
            vz2[index] = vel2[2]
        # integrate 
        else:
            r = get_r([x1[index-1],y1[index-1],z1[index-1]], [x2[index-1],y2[index-1],z2[index-1]])
            F = grav_force(mass1, mass2, r)
            # sets next velocity. 
            # uses the particle's acceleration, last jump, and grav force
            vx1[index] = vx1[index-1]+dt*(acc1-F[0]/mass1)
            vy1[index] = vy1[index-1]+dt*(acc1-F[1]/mass1)
            vz1[index] = vz1[index-1]+dt*(acc1-F[2]/mass1)
            vx2[index] = vx2[index-1]+dt*(acc2+F[0]/mass2)
            vy2[index] = vy2[index-1]+dt*(acc2+F[1]/mass2)
            vz2[index] = vz2[index-1]+dt*(acc2+F[2]/mass2)
            # sets next position. 
            # uses the last jump and next velocity 
            x1[index] = x1[index-1]+dt*vx1[index]
            x2[index] = x2[index-1]+dt*vx2[index]
            y1[index] = y1[index-1]+dt*vy1[index]
            y2[index] = y2[index-1]+dt*vy2[index]
            z1[index] = z1[index-1]+dt*vz1[index]
            z2[index] = z2[index-1]+dt*vz2[index]
    
    return x1,x2,y1,y2,z1,z2,vx1,vx2,vy1,vy2,vz1,vz2

In [3]:
def orbit(ns1, ns2, sep):
    m1=ns1.get_mass()*Ms
    m2=ns2.get_mass()*Ms
    rdist=sep
    M=m1+m2

    r1=m2*rdist/(M)
    r2=rdist*(1-m2/M)
    p1 = np.array([-r1,0,0])
    p2 = np.array([r2,0,0])

    t=0
    dt=1000
    tend = 1e7
    i = 0
    N = int(tend/dt)

    Rcom=np.zeros([N,3])
    Rcom[0]=(m1*p1+m2*p2)/M

    mom1=np.zeros([N,3])
    mom2 = np.zeros([N,3])
    mom1[0] = m1*np.array([0,np.sqrt(G*m2**2/(rdist*M)),0])
    mom2[0] = -1*mom1[0]

    pos1=np.zeros([N,3])
    pos2=np.zeros([N,3])
    pos1[0] = p1
    pos2[0] = p2

    while t<(tend-dt):
        i = i+1
        r=pos2[i-1]-pos1[i-1]
        F2=-G*m1*m2*r/(np.linalg.norm(r)**3)
        mom1[i]=mom1[i-1]-F2*dt
        mom2[i]=mom2[i-1]+F2*dt
        pos1[i]=pos1[i-1]+mom1[i]*dt/m1
        pos2[i]=pos2[i-1]+mom2[i]*dt/m2
        Rcom[i]=(m1*pos1[i]+m2*pos2[i])/M
        t=t+dt
        
    return pos1, pos2, Rcom